In [1]:
# import torch
# import numpy as np
# import random
# seed = 45

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.deterministic = False

import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...
2025-08-26 16:18:42.941039 Epoch [001/002], Step [0001/1092], Loss1: 0.7973 Loss2: 0.6877
2025-08-26 16:19:04.685349 Epoch [001/002], Step [0100/1092], Loss1: 0.3523 Loss2: 0.3103
2025-08-26 16:19:26.781130 Epoch [001/002], Step [0200/1092], Loss1: 0.1578 Loss2: 0.2425
2025-08-26 16:19:48.873062 Epoch [001/002], Step [0300/1092], Loss1: 0.3844 Loss2: 0.4911
2025-08-26 16:20:11.022871 Epoch [001/002], Step [0400/1092], Loss1: 1.0673 Loss2: 0.3901
2025-08-26 16:20:33.173450 Epoch [001/002], Step [0500/1092], Loss1: 0.3129 Loss2: 0.1796
2025-08-26 16:20:55.310919 Epoch [001/002], Step [0600/1092], Loss1: 0.4964 Loss2: 0.4229
2025-08-26 16:21:17.507926 Epoch [001/002], Step [0700/1092], Loss1: 0.4381 Loss2: 0.3452
2025-08-26 16:21:39.629667 Epoch [001/002], Step [0800/1092], Loss1: 0.3156 Loss2: 0.2429
2025-08-26 16:22:01.745971 Epoch [001/002], Step [0900/1092], Loss1: 0.2534 Loss2: 0.1758
2025-08-26 16:22:23.892444 Epoch [001/002], Step [1000/1092], 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [3]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 59157176
Trainable parameters: 59157176
Non-trainable parameters: 0


In [4]:
# test(method='bbsnet') 